## program 1. earning calender for the stock for the day along with MIS/ ASM status

In [2]:
import requests
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill
from datetime import datetime

# Define the URL and headers
url = "https://www.nseindia.com/api/event-calendar"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}

# Initialize session to manage cookies
session = requests.Session()

def fetch_earnings_calendar(url):
    try:
        # Make a GET request to the main page to establish the session
        session.get('https://www.nseindia.com', headers=headers)
        
        # Now, make the request to the API endpoint
        response = session.get(url, headers=headers)
        response.raise_for_status()
        
        earnings_data = response.json()
        
        # Extract relevant fields: symbol, date, purpose
        parsed_data = [
            {
                'symbol': item['symbol'],
                'Result_date': item['date'],
                'Purpose': item['purpose']
            }
            for item in earnings_data
        ]
        
        return pd.DataFrame(parsed_data)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching earnings calendar: {e}")
        return pd.DataFrame()

# Load the pre-saved Excel file and extract the data from 'MIS' sheet
def load_mis_data(filename):
    mis_df = pd.read_excel(filename, sheet_name='MIS')
    return mis_df

# Add the MIS Margin allowed column based on the symbol match
def add_mis_margin(df, mis_df):
    # Use merge to join on 'symbol' and 'Stocks allowed for MIS'
    merged_df = df.merge(mis_df, left_on='symbol', right_on='Stocks allowed for MIS', how='left')
    # Rename the column for clarity
    merged_df.rename(columns={'Margin allowed': 'MIS Margin allowed'}, inplace=True)
    # Fill missing values with "ASM"
    merged_df['MIS Margin allowed'] = merged_df['MIS Margin allowed'].fillna('ASM')
    # Drop the duplicate 'Stocks allowed for MIS' column if needed
    merged_df = merged_df.drop(columns=['Stocks allowed for MIS'])
    return merged_df

def apply_styles(ws, fo_symbols, n=5, threshold_high=3, threshold_mid=1):
    header_font = Font(bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = ws['E2']

    # Identify all pChange columns
    pchange_columns = [cell.column_letter for cell in ws[1] if cell.value and 'pChange' in cell.value]

    # Apply color coding to pChange columns and highlight F&O symbols
    for i, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=2):
        for cell in row:
            if cell.column_letter in pchange_columns:
                if cell.value is not None:
                    if cell.value > 0:
                        if cell.value > threshold_high:
                            cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Bright green
                        elif cell.value > threshold_mid:
                            cell.fill = PatternFill(start_color="66FF66", end_color="66FF66", fill_type="solid")  # Medium green
                        else:
                            cell.fill = PatternFill(start_color="99FF99", end_color="99FF99", fill_type="solid")  # Light green
                    elif cell.value < 0:
                        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

            # Highlight F&O symbols
            if cell.column_letter == 'A' and cell.value in fo_symbols:
                for highlight_cell in row:
                    highlight_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

            # Apply new color coding to every nth row
            if (i - 1) % n == 0:
                cell.fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue color

# Fetch F&O symbols from CSV
def fetch_fo_list_from_csv(file_path):
    try:
        fo_symbols_df = pd.read_csv(file_path)
        print(fo_symbols_df.columns)  # Debugging to check the columns
        fo_symbols_df.columns = fo_symbols_df.columns.str.strip()  # Strip any spaces from column names
        fo_symbols = fo_symbols_df['SYMBOL'].tolist()  # Adjusting based on actual column name
        return fo_symbols
    except Exception as e:
        print(f"Error fetching F&O symbols: {e}")
        return []
        
# Fetch the earnings calendar data
earnings_df = fetch_earnings_calendar(url)
if earnings_df.empty:
    print("No earnings data fetched.")
    exit()

# List available dates (restricted to first five)
unique_dates = earnings_df['Result_date'].unique()[:5]
print("Available dates:")
for i, date in enumerate(unique_dates, start=1):
    print(f"{i}. {date}")

# Ask user to input the dates for specific sheets
selected_dates = []
for i in range(1, 6):
    date_index = int(input(f"Enter the serial number of the date you want to save to Sheet {i}: ")) - 1
    if date_index not in range(5):
        print("Invalid input. Please run the script again and enter valid serial numbers.")
        exit()
    selected_dates.append(unique_dates[date_index])

# Load the MIS data
mis_filename = '/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/fixed_files_getto/rms.xlsx'
mis_data = load_mis_data(mis_filename)

# Load F&O symbols from CSV
fo_csv_path ='/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/fixed_files_getto//MW-SECURITIES-IN-F&O-28-Jul-2024.csv'  # Update with your actual CSV file path
fo_symbols = fetch_fo_list_from_csv(fo_csv_path)

# Create a writer to save data to Excel file with a timestamp
timestamp = datetime.now().strftime("%d%b")
filename = f'{timestamp}_0900_earnings_calendar_with_mis_and_fo.xlsx'
excel_writer = pd.ExcelWriter(filename, engine='openpyxl')
# # Create a writer to save data to Excel file
# excel_writer = pd.ExcelWriter('0900_earnings_calendar_with_mis_and_fo.xlsx', engine='openpyxl')

# Save data for each selected date in a separate sheet
for i, date in enumerate(selected_dates, start=1):
    filtered_df = earnings_df[earnings_df['Result_date'] == date]
    filtered_df = add_mis_margin(filtered_df, mis_data)  # Add MIS margin data
    sheet_name = f"Sheet{i}_{date.replace('-', '_')}"
    filtered_df.to_excel(excel_writer, sheet_name=sheet_name, index=False)

    # Apply styles
    wb = excel_writer.book
    ws = wb[sheet_name]
    apply_styles(ws, fo_symbols)

# Save the Excel file using close() method
excel_writer.close()
print("Data saved to earnings_calendar_with_mis_and_fo.xlsx")


Available dates:
1. 10-Aug-2024
2. 11-Aug-2024
3. 12-Aug-2024
4. 13-Aug-2024
5. 14-Aug-2024
Index(['SYMBOL \n', 'OPEN \n', 'HIGH \n', 'LOW \n', 'PREV. CLOSE \n', 'LTP \n',
       'CHNG \n', '%CHNG \n', 'VOLUME \n(shares)', 'VALUE \n (₹ Crores)',
       '52W H \n', '52W L \n', '30 D   %CHNG \n',
       '365 D % CHNG \n 27-Jul-2023'],
      dtype='object')
Data saved to earnings_calendar_with_mis_and_fo.xlsx


In [4]:
import requests
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill
from datetime import datetime

# Define the URL and headers
base_url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20TOTAL%20MARKET"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}

# Initialize session to manage cookies
session = requests.Session()

def establish_session():
    try:
        # Access the NSE homepage to establish the session
        session.get('https://www.nseindia.com', headers=headers)
    except requests.exceptions.RequestException as e:
        print(f"Error establishing session: {e}")

def fetch_stock_data(symbol):
    try:
        response = session.get(base_url, headers=headers)
        response.raise_for_status()
        stock_data = response.json()['data']
        
        for item in stock_data:
            if item['symbol'] == symbol:
                # Extract the required data fields
                return {
                    'symbol': item['symbol'],
                    'open': item.get('open', ''),
                    'dayHigh': item.get('dayHigh', ''),
                    'dayLow': item.get('dayLow', ''),
                    'lastPrice': item.get('lastPrice', ''),
                    'previousClose': item.get('previousClose', ''),
                    'pChange': item.get('pChange', ''),
                    'totalTradedVolume': item.get('totalTradedVolume', ''),
                    'yearHigh': item.get('yearHigh', ''),
                    'nearWKH': item.get('nearWKH', ''),
                    'perChange30d': item.get('perChange30d', ''),
                    'industry': item['meta'].get('industry', '')
                }
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock data for {symbol}: {e}")
        return None

def apply_styles(ws, fo_symbols=[], n=5, threshold_high=3, threshold_mid=1):
    header_font = Font(bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = ws['E2']

    # Identify all pChange columns
    pchange_columns = [cell.column_letter for cell in ws[1] if cell.value and 'pChange' in cell.value]

    # Apply color coding to pChange columns and highlight F&O symbols
    for i, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=2):
        for cell in row:
            if cell.column_letter in pchange_columns:
                if cell.value is not None:
                    try:
                        cell_value = float(cell.value)
                        if cell_value > 0:
                            if cell_value > threshold_high:
                                cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Bright green
                            elif cell_value > threshold_mid:
                                cell.fill = PatternFill(start_color="66FF66", end_color="66FF66", fill_type="solid")  # Medium green
                            else:
                                cell.fill = PatternFill(start_color="99FF99", end_color="99FF99", fill_type="solid")  # Light green
                        elif cell_value < 0:
                            cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
                    except ValueError:
                        # If the cell value cannot be converted to float, skip it
                        continue

            # Highlight F&O symbols
            if cell.column_letter == 'A' and cell.value in fo_symbols:
                for highlight_cell in row:
                    highlight_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

            # Apply new color coding to every nth row
            if (i - 1) % n == 0:
                cell.fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue color

def update_excel_sheets(file_path):
    # Load the workbook and get all sheet names
    wb = load_workbook(file_path)
    sheets = wb.sheetnames

    # Prompt user to select the sheet for which data needs to be fetched
    print("Available sheets:")
    for i, sheet in enumerate(sheets, start=1):
        print(f"{i}. {sheet}")
    
    sheet_index = int(input("Enter the serial number of the sheet you want to update: ")) - 1
    if sheet_index not in range(len(sheets)):
        print("Invalid input. Please run the script again and enter a valid serial number.")
        return
    
    selected_sheet_name = sheets[sheet_index]

    # Create a new workbook to save the updated data
    new_wb = Workbook()
    new_wb.remove(new_wb.active)  # Remove the default sheet

    ws = wb[selected_sheet_name]
    df = pd.DataFrame(ws.values)

    # Extract the symbol column (assuming it's in the first column)
    symbols = df[0].dropna().values[1:]  # Skip header

    # List to store new rows
    new_rows = []

    for symbol in symbols:
        stock_data = fetch_stock_data(symbol)
        if stock_data:
            new_row = [
                stock_data['symbol'],
                stock_data['open'],
                stock_data['dayHigh'],
                stock_data['dayLow'],
                stock_data['lastPrice'],
                stock_data['previousClose'],
                stock_data['pChange'],
                stock_data['totalTradedVolume'],
                stock_data['yearHigh'],
                stock_data['nearWKH'],
                stock_data['perChange30d'],
                stock_data['industry']
            ]
        else:
            # Maintain data integrity by adding an empty row if data is not found
            new_row = [''] * 12
        new_rows.append(new_row)

    # Convert to DataFrame
    new_df = pd.DataFrame(new_rows, columns=[
        'symbol', 'open', 'dayHigh', 'dayLow', 'lastPrice', 'previousClose',
        'pChange', 'totalTradedVolume', 'yearHigh', 'nearWKH', 'perChange30d', 'industry'
    ])

    # Append to the right of existing data
    for r_idx, row in enumerate(dataframe_to_rows(new_df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, len(df.columns) + 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    # Add the sheet with updated data to the new workbook
    new_ws = new_wb.create_sheet(selected_sheet_name)
    for row in ws.iter_rows(values_only=True):
        new_ws.append(row)

    # Apply styles to the newly appended data
    apply_styles(new_ws, fo_symbols=[])

    # Generate the new file path
    timestamp = datetime.now().strftime("%d%b%Y_%H%M")
    new_file_path = f"{file_path.split('.xlsx')[0]}_updated_{timestamp}.xlsx"

    # Save the new workbook
    new_wb.save(new_file_path)
    print(f"Data successfully saved to {new_file_path}")

# Establish the session
establish_session()

# Specify the Excel file path
file_path = '/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/09Aug_0900_earnings_calendar_with_mis_and_fo.xlsx'

# Update the selected sheet and save the new workbook
update_excel_sheets(file_path)


Available sheets:
1. Sheet1_14_Aug_2024
2. Sheet2_10_Aug_2024
3. Sheet3_11_Aug_2024
4. Sheet4_12_Aug_2024
5. Sheet5_13_Aug_2024
6. Sheet6_14_Aug_2024
Data successfully saved to /Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/09Aug_0900_earnings_calendar_with_mis_and_fo_updated_09Aug2024_0318.xlsx


In [10]:
import requests
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime

# Define the URL and headers
base_url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20TOTAL%20MARKET"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}

# Initialize session to manage cookies
session = requests.Session()

def establish_session():
    try:
        # Access the NSE homepage to establish the session
        session.get('https://www.nseindia.com', headers=headers)
    except requests.exceptions.RequestException as e:
        print(f"Error establishing session: {e}")

def fetch_stock_data(symbol):
    try:
        response = session.get(base_url, headers=headers)
        response.raise_for_status()
        stock_data = response.json()['data']
        
        for item in stock_data:
            if item['symbol'] == symbol:
                # Extract the required data fields
                return {
                    'symbol': item['symbol'],
                    'open': item.get('open', ''),
                    'dayHigh': item.get('dayHigh', ''),
                    'dayLow': item.get('dayLow', ''),
                    'lastPrice': item.get('lastPrice', ''),
                    'previousClose': item.get('previousClose', ''),
                    'pChange': item.get('pChange', ''),
                    'totalTradedVolume': item.get('totalTradedVolume', ''),
                    'yearHigh': item.get('yearHigh', ''),
                    'nearWKH': item.get('nearWKH', ''),
                    'perChange30d': item.get('perChange30d', ''),
                    'industry': item['meta'].get('industry', '')
                }
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock data for {symbol}: {e}")
        return None

def update_excel_sheet(file_path, sheet_name):
    # Load the workbook and the selected sheet
    wb = load_workbook(file_path)
    if sheet_name not in wb.sheetnames:
        print(f"Sheet {sheet_name} not found in the workbook.")
        return
    
    ws = wb[sheet_name]
    df = pd.DataFrame(ws.values)

    # Extract the symbol column (assuming it's in the first column)
    symbols = df[0].dropna().values[1:]  # Skip header

    # List to store new rows
    new_rows = []

    for symbol in symbols:
        stock_data = fetch_stock_data(symbol)
        if stock_data:
            new_row = [
                stock_data['symbol'],
                stock_data['open'],
                stock_data['dayHigh'],
                stock_data['dayLow'],
                stock_data['lastPrice'],
                stock_data['previousClose'],
                stock_data['pChange'],
                stock_data['totalTradedVolume'],
                stock_data['yearHigh'],
                stock_data['nearWKH'],
                stock_data['perChange30d'],
                stock_data['industry']
            ]
            new_rows.append(new_row)

    # Convert to DataFrame
    new_df = pd.DataFrame(new_rows, columns=[
        'symbol', 'open', 'dayHigh', 'dayLow', 'lastPrice', 'previousClose',
        'pChange', 'totalTradedVolume', 'yearHigh', 'nearWKH', 'perChange30d', 'industry'
    ])

    # Append to the right of existing data
    for r_idx, row in enumerate(dataframe_to_rows(new_df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, len(df.columns) + 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    # Save the updated workbook
    wb.save(file_path)
    print(f"Data successfully appended to {file_path} in sheet {sheet_name}.")

# Establish the session
establish_session()


# Specify the Excel file path and the sheet to update
file_path = '/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/database/Aug_24/7 Aug 24/07Aug_0900_earnings_calendar_with_mis_and_fo.xlsx'
sheet_name = 'Sheet2_09_Aug_2024'  # Update this to the actual sheet name

update_excel_sheet(file_path, sheet_name)


Data successfully appended to /Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/database/Aug_24/7 Aug 24/07Aug_0900_earnings_calendar_with_mis_and_fo.xlsx in sheet Sheet2_09_Aug_2024.


In [15]:
import requests
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
from openpyxl.styles import Font, PatternFill

# Define the URL and headers
base_url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20TOTAL%20MARKET"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}

# Initialize session to manage cookies
session = requests.Session()

def establish_session():
    try:
        # Access the NSE homepage to establish the session
        session.get('https://www.nseindia.com', headers=headers)
    except requests.exceptions.RequestException as e:
        print(f"Error establishing session: {e}")

def fetch_stock_data(symbol):
    try:
        response = session.get(base_url, headers=headers)
        response.raise_for_status()
        stock_data = response.json()['data']
        
        for item in stock_data:
            if item['symbol'] == symbol:
                # Extract the required data fields
                return {
                    'symbol': item['symbol'],
                    'open': item.get('open', ''),
                    'dayHigh': item.get('dayHigh', ''),
                    'dayLow': item.get('dayLow', ''),
                    'lastPrice': item.get('lastPrice', ''),
                    'previousClose': item.get('previousClose', ''),
                    'pChange': item.get('pChange', ''),
                    'totalTradedVolume': item.get('totalTradedVolume', ''),
                    'yearHigh': item.get('yearHigh', ''),
                    'nearWKH': item.get('nearWKH', ''),
                    'perChange30d': item.get('perChange30d', ''),
                    'industry': item['meta'].get('industry', '')
                }
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock data for {symbol}: {e}")
        return None

def apply_styles(ws, fo_symbols, n=5, threshold_high=3, threshold_mid=1):
    header_font = Font(bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = ws['E2']

    # Identify all pChange columns
    pchange_columns = [cell.column_letter for cell in ws[1] if cell.value and 'pChange' in cell.value]

    # Apply color coding to pChange columns and highlight F&O symbols
    for i, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=2):
        for cell in row:
            if cell.column_letter in pchange_columns:
                if cell.value is not None:
                    if cell.value > 0:
                        if cell.value > threshold_high:
                            cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Bright green
                        elif cell.value > threshold_mid:
                            cell.fill = PatternFill(start_color="66FF66", end_color="66FF66", fill_type="solid")  # Medium green
                        else:
                            cell.fill = PatternFill(start_color="99FF99", end_color="99FF99", fill_type="solid")  # Light green
                    elif cell.value < 0:
                        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

            # Highlight F&O symbols
            if cell.column_letter == 'A' and cell.value in fo_symbols:
                for highlight_cell in row:
                    highlight_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

            # Apply new color coding to every nth row
            if (i - 1) % n == 0:
                cell.fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue color

def update_excel_sheet(file_path):
    # Load the workbook and list sheet names
    wb = load_workbook(file_path)
    sheets = wb.sheetnames
    print("Available sheets:")
    for i, sheet in enumerate(sheets, start=1):
        print(f"{i}. {sheet}")
    
    # Prompt user to select a sheet
    sheet_index = int(input("Enter the serial number of the sheet you want to update: ")) - 1
    if sheet_index not in range(len(sheets)):
        print("Invalid input. Please run the script again and enter a valid serial number.")
        return
    
    sheet_name = sheets[sheet_index]
    ws = wb[sheet_name]
    df = pd.DataFrame(ws.values)

    # Extract the symbol column (assuming it's in the first column)
    symbols = df[0].dropna().values[1:]  # Skip header

    # List to store new rows
    new_rows = []

    for symbol in symbols:
        stock_data = fetch_stock_data(symbol)
        if stock_data:
            new_row = [
                stock_data['symbol'],
                stock_data['open'],
                stock_data['dayHigh'],
                stock_data['dayLow'],
                stock_data['lastPrice'],
                stock_data['previousClose'],
                stock_data['pChange'],
                stock_data['totalTradedVolume'],
                stock_data['yearHigh'],
                stock_data['nearWKH'],
                stock_data['perChange30d'],
                stock_data['industry']
            ]
        else:
            # Maintain data integrity by adding an empty row if data is not found
            new_row = [''] * 12
        new_rows.append(new_row)

    # Convert to DataFrame
    new_df = pd.DataFrame(new_rows, columns=[
        'symbol', 'open', 'dayHigh', 'dayLow', 'lastPrice', 'previousClose',
        'pChange', 'totalTradedVolume', 'yearHigh', 'nearWKH', 'perChange30d', 'industry'
    ])

    # Append to the right of existing data
    for r_idx, row in enumerate(dataframe_to_rows(new_df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, len(df.columns) + 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    # Apply styles to the newly appended data
    apply_styles(ws, fo_symbols=[])

    # Save the updated workbook
    wb.save(file_path)
    print(f"Data successfully appended to {file_path} in sheet {sheet_name}.")

# Establish the session
establish_session()

# Specify the Excel file path
file_path = '/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/database/Aug_24/8 Aug 24/08Aug_0900_earnings_calendar_with_mis_and_fo.xlsx'

update_excel_sheet(file_path)


Available sheets:
1. Sheet1_09_Aug_2024
2. Sheet2_10_Aug_2024
3. Sheet3_11_Aug_2024
4. Sheet4_12_Aug_2024
5. Sheet5_13_Aug_2024


TypeError: '>' not supported between instances of 'str' and 'int'

In [14]:
import requests
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime

# Define the URLs and headers
calendar_url = "https://www.nseindia.com/api/event-calendar"
stock_data_url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20TOTAL%20MARKET"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}

# Initialize session to manage cookies
session = requests.Session()

def fetch_earnings_calendar(url):
    try:
        session.get('https://www.nseindia.com', headers=headers)
        response = session.get(url, headers=headers)
        response.raise_for_status()
        earnings_data = response.json()
        parsed_data = [
            {
                'symbol': item['symbol'],
                'Result_date': item['date'],
                'Purpose': item['purpose']
            }
            for item in earnings_data
        ]
        return pd.DataFrame(parsed_data)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching earnings calendar: {e}")
        return pd.DataFrame()

def load_mis_data(filename):
    mis_df = pd.read_excel(filename, sheet_name='MIS')
    return mis_df

def add_mis_margin(df, mis_df):
    merged_df = df.merge(mis_df, left_on='symbol', right_on='Stocks allowed for MIS', how='left')
    merged_df.rename(columns={'Margin allowed': 'MIS Margin allowed'}, inplace=True)
    merged_df['MIS Margin allowed'] = merged_df['MIS Margin allowed'].fillna('ASM')
    merged_df = merged_df.drop(columns=['Stocks allowed for MIS'])
    return merged_df

def apply_styles(ws, fo_symbols, n=5, threshold_high=3, threshold_mid=1):
    header_font = Font(bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = ws['E2']

    pchange_columns = [cell.column_letter for cell in ws[1] if cell.value and 'pChange' in cell.value]

    for i, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), start=2):
        for cell in row:
            if cell.column_letter in pchange_columns:
                if cell.value is not None:
                    if cell.value > 0:
                        if cell.value > threshold_high:
                            cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
                        elif cell.value > threshold_mid:
                            cell.fill = PatternFill(start_color="66FF66", end_color="66FF66", fill_type="solid")
                        else:
                            cell.fill = PatternFill(start_color="99FF99", end_color="99FF99", fill_type="solid")
                    elif cell.value < 0:
                        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

            if cell.column_letter == 'A' and cell.value in fo_symbols:
                for highlight_cell in row:
                    highlight_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

            if (i - 1) % n == 0:
                cell.fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")

def fetch_fo_list_from_csv(file_path):
    try:
        fo_symbols_df = pd.read_csv(file_path)
        fo_symbols_df.columns = fo_symbols_df.columns.str.strip()
        fo_symbols = fo_symbols_df['SYMBOL'].tolist()
        return fo_symbols
    except Exception as e:
        print(f"Error fetching F&O symbols: {e}")
        return []

def fetch_stock_data(symbol):
    try:
        response = session.get(stock_data_url, headers=headers)
        response.raise_for_status()
        stock_data = response.json()['data']

        for item in stock_data:
            if item['symbol'] == symbol:
                return {
                    'symbol': item['symbol'],
                    'open': item.get('open', ''),
                    'dayHigh': item.get('dayHigh', ''),
                    'dayLow': item.get('dayLow', ''),
                    'lastPrice': item.get('lastPrice', ''),
                    'previousClose': item.get('previousClose', ''),
                    'pChange': item.get('pChange', ''),
                    'totalTradedVolume': item.get('totalTradedVolume', ''),
                    'yearHigh': item.get('yearHigh', ''),
                    'nearWKH': item.get('nearWKH', ''),
                    'perChange30d': item.get('perChange30d', ''),
                    'industry': item['meta'].get('industry', '')
                }
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock data for {symbol}: {e}")
        return None

def update_excel_sheet(file_path, sheet_name, fo_symbols):
    wb = load_workbook(file_path)
    if sheet_name not in wb.sheetnames:
        print(f"Sheet {sheet_name} not found in the workbook.")
        return
    
    ws = wb[sheet_name]
    df = pd.DataFrame(ws.values)

    symbols = df[0].dropna().values[1:]

    new_rows = []

    for symbol in symbols:
        stock_data = fetch_stock_data(symbol)
        if stock_data:
            new_row = [
                stock_data['symbol'],
                stock_data['open'],
                stock_data['dayHigh'],
                stock_data['dayLow'],
                stock_data['lastPrice'],
                stock_data['previousClose'],
                stock_data['pChange'],
                stock_data['totalTradedVolume'],
                stock_data['yearHigh'],
                stock_data['nearWKH'],
                stock_data['perChange30d'],
                stock_data['industry']
            ]
        else:
            new_row = [symbol] + [''] * 11
        new_rows.append(new_row)

    new_df = pd.DataFrame(new_rows, columns=[
        'symbol', 'open', 'dayHigh', 'dayLow', 'lastPrice', 'previousClose',
        'pChange', 'totalTradedVolume', 'yearHigh', 'nearWKH', 'perChange30d', 'industry'
    ])

    for r_idx, row in enumerate(dataframe_to_rows(new_df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, len(df.columns) + 1):
            ws.cell(row=r_idx, column=c_idx, value=value)

    apply_styles(ws, fo_symbols)
    wb.save(file_path)
    print(f"Data successfully appended to {file_path} in sheet {sheet_name}.")

# Fetch the earnings calendar data
earnings_df = fetch_earnings_calendar(calendar_url)
if earnings_df.empty:
    print("No earnings data fetched.")
    exit()

unique_dates = earnings_df['Result_date'].unique()[:5]
print("Available dates:")
for i, date in enumerate(unique_dates, start=1):
    print(f"{i}. {date}")

selected_dates = []
for i in range(0, 5):  # Only ask for 5 dates
    date_index = int(input(f"Enter the serial number of the date you want to save to Sheet {i+1}: ")) - 1
    if date_index not in range(5):
        print("Invalid input. Please run the script again and enter valid serial numbers.")
        exit()
    selected_dates.append(unique_dates[date_index])

mis_filename = '/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/fixed_files_getto/rms.xlsx'
mis_data = load_mis_data(mis_filename)

fo_csv_path = '/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/fixed_files_getto/MW-SECURITIES-IN-F&O-28-Jul-2024.csv'
fo_symbols = fetch_fo_list_from_csv(fo_csv_path)

timestamp = datetime.now().strftime("%d%b")
filename = f'/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/database/Aug_24/8 Aug 24/{timestamp}_0900_earnings_calendar_with_mis_and_fo.xlsx'
excel_writer = pd.ExcelWriter(filename, engine='openpyxl')

# Save data for each selected date in a separate sheet
for i, date in enumerate(selected_dates, start=1):
    filtered_df = earnings_df[earnings_df['Result_date'] == date]
    filtered_df = add_mis_margin(filtered_df, mis_data)
    sheet_name = f"Sheet{i}_{date.replace('-', '_')}"
    filtered_df.to_excel(excel_writer, sheet_name=sheet_name, index=False)

    # Apply styles to the existing data
    wb = excel_writer.book
    ws = wb[sheet_name]
    apply_styles(ws, fo_symbols)

# Save the Excel file first
excel_writer.close()

# Now reopen the file and append stock data
for i, date in enumerate(selected_dates, start=1):
    sheet_name = f"Sheet{i}_{date.replace('-', '_')}"
    update_excel_sheet(filename, sheet_name, fo_symbols)

print("Data saved to earnings_calendar_with_mis_and_fo.xlsx")


Available dates:
1. 10-Aug-2024
2. 11-Aug-2024
3. 12-Aug-2024
4. 13-Aug-2024
5. 14-Aug-2024


TypeError: '>' not supported between instances of 'str' and 'int'

## Program 2 : Nifty total and FnO pre market status for the day with MIS margins and IEP values > Pr.Close

In [5]:
## 1. Program to fetch Nifty Total Pre-Open data and Nifty FO Pre-Open data.
## 2. Save the data to Excel files.
## 3. Update the data in the Excel files.
## 4. Apply color coding to the data in the Excel files.
## 5. Apply styles to the data in the Excel files.
## 6. Update the filtered data in the Excel files based on the pChange threshold.


import requests
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill
from datetime import datetime
import os
import colorsys
from bs4 import BeautifulSoup

# Initialize session object
session = requests.Session()
headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
}

def get_cookie():
    url = 'https://www.nseindia.com'
    response = session.get(url, headers=headers)
    cookies = response.cookies.get_dict()
    cookie_str = '; '.join([f"{key}={value}" for key, value in cookies.items()])
    return cookie_str

def fetch_data(url):
    retries = 3
    for attempt in range(retries):
        headers['cookie'] = get_cookie()
        try:
            response = session.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            if response.status_code in [401, 403]:
                print(f"Attempt {attempt+1}: Cookie expired. Fetching a new cookie...")
            else:
                print(f"HTTP Error {response.status_code}: {response.text}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
            break
    return None

def clean_value(value):
    if isinstance(value, str):
        return float(value.replace(',', ''))
    return float(value)

def extract_data(positions):
    data_list = []
    for item in positions['data']:
        metadata = item['metadata']
        detail = item.get('detail', {}).get('preOpenMarket', {})
        symbol = metadata['symbol']
        ltp = clean_value(metadata.get('lastPrice', 0))
        p_change = round(clean_value(metadata.get('pChange', 0)), 3)
        iep = clean_value(detail.get('IEP', 0))
        day_low = clean_value(metadata.get('yearLow', 0))
        day_high = clean_value(metadata.get('yearHigh', 0))
        previous_close = clean_value(metadata.get('previousClose', 0))
        total_value_crores = round(clean_value(metadata.get('totalTurnover', 0)) / 1e7, 3)
        data_list.append([symbol, ltp, p_change, iep, day_low, day_high, previous_close, total_value_crores])
    df = pd.DataFrame(data_list, columns=['symbol', 'LTP', 'pChange', 'IEP', 'D_Low', 'D_High', 'Pr.Close', 'Total_Value_Crores'])
    return df

def append_timestamp_columns(existing_df, new_df):
    timestamp = datetime.now().strftime('%H:%M:%S')
    new_df_unique = new_df.drop_duplicates(subset='symbol')
    existing_df[f'LTP_{timestamp}'] = existing_df['symbol'].map(new_df_unique.set_index('symbol')['LTP'])
    existing_df[f'pChange_{timestamp}'] = existing_df['symbol'].map(new_df_unique.set_index('symbol')['pChange'])
    return existing_df

def calculate_pchange_diff(existing_df, timestamp):
    pchange_columns = [col for col in existing_df.columns if col.startswith('pChange_') and col != f'pChange_{timestamp}']
    
    if pchange_columns:
        pchange_columns.sort()
        previous_pchange_col = pchange_columns[-1]
        
        existing_df[f'pChange_Diff_{timestamp}'] = existing_df[f'pChange_{timestamp}'] - existing_df[previous_pchange_col]
    
    return existing_df

def apply_color_coding(ws):
    for col in ws.iter_cols(min_col=1, max_col=ws.max_column):
        if col[0].value and 'pChange' in col[0].value:
            values = [cell.value for cell in col[1:] if cell.value is not None and not pd.isna(cell.value)]
            if values:
                min_val = min(values)
                max_val = max(values)
                for cell in col[1:]:
                    if cell.value is not None and not pd.isna(cell.value):
                        norm_value = (cell.value - min_val) / (max_val - min_val) if max_val != min_val else 0
                        if cell.value > 0:
                            hue = 120
                            saturation = norm_value * 100
                            lightness = 50
                        else:
                            hue = 0
                            saturation = abs(norm_value) * 100
                            lightness = 50
                        rgb_color = colorsys.hls_to_rgb(hue / 360, lightness / 100, saturation / 100)
                        hex_color = '{:02X}{:02X}{:02X}'.format(int(rgb_color[0] * 255), int(rgb_color[1] * 255), int(rgb_color[2] * 255))
                        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
                        
    for col in ws.iter_cols(min_col=1, max_col=ws.max_column):
        if col[0].value == 'Total_Value_Crores':
            values = [cell.value for cell in col[1:] if cell.value is not None and not pd.isna(cell.value)]
            if values:
                min_val = min(values)
                max_val = max(values)
                for cell in col[1:]:
                    if cell.value is not None and not pd.isna(cell.value):
                        norm_value = (cell.value - min_val) / (max_val - min_val) if max_val != min_val else 0
                        hue = 240
                        saturation = norm_value * 100
                        lightness = 50
                        rgb_color = colorsys.hls_to_rgb(hue / 360, lightness / 100, saturation / 100)
                        hex_color = '{:02X}{:02X}{:02X}'.format(int(rgb_color[0] * 255), int(rgb_color[1] * 255), int(rgb_color[2] * 255))
                        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
        
        # Apply color coding for 'MIS Margin allowed' column
        if col[0].value == 'MIS Margin allowed':
            for cell in col[1:]:
                if cell.value == '5X':
                    cell.fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")

def apply_styles(ws):
    header_font = Font(bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
        cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    ws.auto_filter.ref = ws.dimensions
    ws.freeze_panes = ws['B2']

    apply_color_coding(ws)

def save_to_excel(df, filename, sheet_name):
    if os.path.exists(filename):
        wb = load_workbook(filename)
        if sheet_name in wb.sheetnames:
            ws = wb[sheet_name]
            data = ws.values
            columns = next(data)[0:]
            existing_df = pd.DataFrame(data, columns=columns)

            timestamp = datetime.now().strftime('%H:%M:%S')
            existing_df = append_timestamp_columns(existing_df, df)
            existing_df = calculate_pchange_diff(existing_df, timestamp)

            for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
                for cell in row:
                    cell.value = None
        else:
            ws = wb.create_sheet(title=sheet_name)
            existing_df = df
            for col_num, column_title in enumerate(existing_df.columns, 1):
                ws.cell(row=1, column=col_num, value=column_title)
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = sheet_name
        existing_df = df
        for col_num, column_title in enumerate(existing_df.columns, 1):
            ws.cell(row=1, column=col_num, value=column_title)

    for col_num, column_title in enumerate(existing_df.columns, 1):
        ws.cell(row=1, column=col_num, value=column_title)

    for r_idx, row in existing_df.iterrows():
        for c_idx, value in enumerate(row):
            ws.cell(row=r_idx + 2, column=c_idx + 1, value=value)

    apply_styles(ws)
    wb.save(filename)
    print(f"Data updated in {filename} in sheet {sheet_name}")

def update_filtered_data(df, filename, sheet_name, filter_condition, pchange_threshold):
    filtered_df = df[filter_condition & (df['pChange'] > pchange_threshold)]

    if os.path.exists(filename):
        wb = load_workbook(filename)
        if sheet_name in wb.sheetnames:
            ws_filtered = wb[sheet_name]
            data = ws_filtered.values
            columns = next(data)[0:]
            existing_df = pd.DataFrame(data, columns=columns)
            timestamp = datetime.now().strftime('%H:%M:%S')
            existing_df = append_timestamp_columns(existing_df, filtered_df)
            existing_df = calculate_pchange_diff(existing_df, timestamp)

            for row in ws_filtered.iter_rows(min_row=2, max_row=ws_filtered.max_row, min_col=1, max_col=ws_filtered.max_column):
                for cell in row:
                    cell.value = None
        else:
            ws_filtered = wb.create_sheet(title=sheet_name)
            existing_df = filtered_df
            for col_num, column_title in enumerate(existing_df.columns, 1):
                ws_filtered.cell(row=1, column=col_num, value=column_title)
    else:
        wb = Workbook()
        ws_filtered = wb.active
        ws_filtered.title = sheet_name
        existing_df = filtered_df
        for col_num, column_title in enumerate(existing_df.columns, 1):
            ws_filtered.cell(row=1, column=col_num, value=column_title)

    for r_idx, row in existing_df.iterrows():
        for c_idx, value in enumerate(row):
            ws_filtered.cell(row=r_idx + 2, column=c_idx + 1, value=value)

    apply_styles(ws_filtered)
    wb.save(filename)
    print(f"{sheet_name} data updated in {filename}")

def fetch_and_save_nifty_total_pre_open(pchange_threshold):
    nifty_url = 'https://www.nseindia.com/api/market-data-pre-open?key=ALL'
    while True:
        nifty_data = fetch_data(nifty_url)
        if nifty_data:
            df = extract_data(nifty_data)
            print(df.head())

            # Define the Excel filename
            excel_filename = f'{datetime.now().strftime("%d%b_%H:%M")}_0908_1_nifty_Total_pre_open.xlsx'

            # Save the main DataFrame to the Nifty Data sheet
            save_to_excel(df, excel_filename, 'Nifty Data')

            # Update the filtered data sheets
            update_filtered_data(df, excel_filename, 'IEP_Eql_Low', df['IEP'] == df['D_Low'], pchange_threshold)
            update_filtered_data(df, excel_filename, 'IEP>Pr.Close', df['IEP'] > df['Pr.Close'], pchange_threshold)
            update_filtered_data(df, excel_filename, 'IEP=L>Pr.Cls', (df['IEP'] == df['D_Low']) & (df['IEP'] > df['Pr.Close']), pchange_threshold)
            break
        else:
            print("Failed to fetch Nifty Total Pre-Open data, retrying...")

def fetch_and_save_nifty_fo_pre_open(pchange_threshold):
    fo_url = 'https://www.nseindia.com/api/market-data-pre-open?key=FO'
    while True:
        fo_data = fetch_data(fo_url)
        if fo_data:
            df_fo = extract_data(fo_data)
            print(df_fo.head())

            # Define the new Excel filename
            fo_excel_filename = f'{datetime.now().strftime("%d%b_%H:%M")}_0908_1_nifty_fno_pre_open.xlsx'

            # Save the FO DataFrame to a new sheet
            save_to_excel(df_fo, fo_excel_filename, 'FO Data')

            # Update the filtered data sheets for the new URL
            update_filtered_data(df_fo, fo_excel_filename, 'IEP_Eql_Low', df_fo['IEP'] == df_fo['D_Low'], pchange_threshold)
            update_filtered_data(df_fo, fo_excel_filename, 'IEP>Pr.Close', df_fo['IEP'] > df_fo['Pr.Close'], pchange_threshold)
            update_filtered_data(df_fo, fo_excel_filename, 'IEP=L>Pr.Cls', (df_fo['IEP'] == df_fo['D_Low']) & (df_fo['IEP'] > df_fo['Pr.Close']), pchange_threshold)
            break
        else:
            print("Failed to fetch Nifty FO Pre-Open data, retrying...")

# Load the pre-saved Excel file and extract the data from 'MIS' sheet
def load_mis_data(filename):
    mis_df = pd.read_excel(filename, sheet_name='MIS')
    return mis_df

def add_mis_margin(df, mis_df):
    # Use merge to join on 'symbol' and 'Stocks allowed for MIS'
    merged_df = df.merge(mis_df, left_on='symbol', right_on='Stocks allowed for MIS', how='left')
    # Rename the column for clarity
    merged_df.rename(columns={'Margin allowed': 'MIS Margin allowed'}, inplace=True)
    # Fill missing values with "ASM"
    merged_df['MIS Margin allowed'] = merged_df['MIS Margin allowed'].fillna('ASM')
    # Drop the duplicate 'Stocks allowed for MIS' column if needed
    merged_df = merged_df.drop(columns=['Stocks allowed for MIS'])
    return merged_df

# Adding MIS Margin allowed to the output files
def add_mis_to_output_files(output_filename):
    wb = load_workbook(output_filename)
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        data = ws.values
        columns = next(data)[0:]
        existing_df = pd.DataFrame(data, columns=columns)
        existing_df = add_mis_margin(existing_df, mis_data)

        # Write the header
        for col_num, column_title in enumerate(existing_df.columns, 1):
            ws.cell(row=1, column=col_num, value=column_title)

        # Write the data
        for r_idx, row in existing_df.iterrows():
            for c_idx, value in enumerate(row):
                ws.cell(row=r_idx + 2, column=c_idx + 1, value=value)
        apply_styles(ws)
    wb.save(output_filename)
    print(f"MIS Margin allowed column added to {output_filename}")

# Prompt the user to input the pChange threshold
pchange_threshold = float(input("Enter the pChange threshold: "))

# Fetch and save Nifty Total Pre-Open data
fetch_and_save_nifty_total_pre_open(pchange_threshold)

# Fetch and save Nifty FO Pre-Open data
fetch_and_save_nifty_fo_pre_open(pchange_threshold)

# Load the pre-saved Excel file and extract the MIS data
mis_filename = '/Users/tjzoomac/tj_nse/ai_market/01_nifty_live/002/fixed_files_getto/rms.xlsx'
mis_data = load_mis_data(mis_filename)

# Update the output files with MIS Margin allowed column
output_files = [
    f'{datetime.now().strftime("%d%b_%H:%M")}_0908_1_nifty_Total_pre_open.xlsx',
    f'{datetime.now().strftime("%d%b_%H:%M")}_0908_1_nifty_fno_pre_open.xlsx'
]

for output_file in output_files:
    add_mis_to_output_files(output_file)

       symbol      LTP  pChange      IEP   D_Low   D_High  Pr.Close  \
0   NIRMAN-RE   109.80   21.058   109.80   73.05   115.00     90.70   
1   GATECH-RE     0.00   20.000     0.06    0.04     0.15      0.05   
2      SAKUMA     7.69   18.673     7.69   14.16    39.29     32.39   
3      3PLAND    55.54   18.574    55.54   19.25    46.84     46.84   
4  GRWRHITECH  3400.00   17.428  3400.00  893.00  2895.40   2895.40   

   Total_Value_Crores  
0               0.005  
1               0.000  
2               0.256  
3               0.306  
4              17.462  
Data updated in 09Aug_09:10_0908_1_nifty_Total_pre_open.xlsx in sheet Nifty Data
IEP_Eql_Low data updated in 09Aug_09:10_0908_1_nifty_Total_pre_open.xlsx
IEP>Pr.Close data updated in 09Aug_09:10_0908_1_nifty_Total_pre_open.xlsx
IEP=L>Pr.Cls data updated in 09Aug_09:10_0908_1_nifty_Total_pre_open.xlsx
      symbol      LTP  pChange      IEP   D_Low   D_High  Pr.Close  \
0  EICHERMOT  4805.65    4.998  4805.65  3274.9  5058.90 